## 第一步：环境配置与数据读取

In [2]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
# import open3d as o3d  # 推荐安装 open3d 用于3D可视化

# 配置路径
DATA_ROOT = "/home/xzy/datasets/Rellis-3D" 
SEQ = "00004"
FRAME_ID = "000000"  # 示例帧

# 路径拼接 (根据 Rellis 目录结构)
# Rellis 通常结构: 00004/os1_cloud_node_kitti_bin/000000.bin
bin_path = os.path.join(DATA_ROOT, SEQ, 'os1_cloud_node_kitti_bin', f'{FRAME_ID}.bin')
img_path = os.path.join(DATA_ROOT, SEQ, 'pylon_camera_node', f'frame{FRAME_ID}-...jpg') # 需补全实际文件名

# 读取点云 (x, y, z, intensity)
points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
print(f"Point cloud shape: {points.shape}")

Point cloud shape: (131072, 4)


## 第二步：理解相机与雷达的关系（外参标定）

## 第三步：解析 poses.txt 与世界坐标系

坐标轴位姿可视化：红绿蓝 x y z

使用 o3d.geometry.TriangleMesh.create_sphere 在轨迹两端放置了球体。

起点：绿色 [0, 1, 0]，代表 "Go/Start"。

终点：红色 [1, 0, 0]，代表 "Stop/End"。

poses.txt立即为雷达系位姿（前x左y上z），可视化发现车子是倒开。

In [8]:
import open3d as o3d
import numpy as np



def parse_poses(filename):
    poses = []
    with open(filename, 'r') as f:
        for line in f:
            values = [float(v) for v in line.strip().split()]
            pose = np.eye(4)
            pose[:3, :4] = np.array(values).reshape(3, 4)
            poses.append(pose)
    return poses

pose_file = os.path.join(DATA_ROOT, SEQ, 'poses.txt')
poses = parse_poses(pose_file)

def visualize_trajectory_open3d(poses, visual_interval=10):
    """
    poses: 4x4 变换矩阵列表
    visual_interval: 每隔多少帧画一个坐标轴 (避免太密集)
    """
    geometries = []

    # 1. 创建轨迹线 (LineSet)
    # 提取所有平移向量 (位置)
    points = [p[:3, 3] for p in poses]
    
    # 定义线段连接关系: 0-1, 1-2, 2-3 ...
    lines = [[i, i+1] for i in range(len(points)-1)]
    
    # 创建 LineSet 对象
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.paint_uniform_color([0, 0, 0]) # 黑色轨迹线
    geometries.append(line_set)

    # 2. 创建坐标轴 (Coordinate Frames)
    # 红色=X(前), 绿色=Y(左), 蓝色=Z(上)
    for i in range(0, len(poses), visual_interval):
        # 创建一个小坐标轴，size 控制大小
        mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2.0, origin=[0, 0, 0])
        
        # 将坐标轴变换到对应的位姿位置
        mesh_frame.transform(poses[i])
        geometries.append(mesh_frame)

    # 3. 特殊处理：起点 (Start Node)
    # 绿色球体 + 较大的坐标轴
    start_pose = poses[0]
    start_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.8)
    start_sphere.paint_uniform_color([0, 1, 0])  # 纯绿色
    start_sphere.translate(start_pose[:3, 3])
    
    start_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3.0, origin=[0, 0, 0])
    start_frame.transform(start_pose)
    
    geometries.extend([start_sphere, start_frame])

    # 4. 特殊处理：终点 (End Node)
    # 红色球体 + 较大的坐标轴
    end_pose = poses[-1]
    end_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.8)
    end_sphere.paint_uniform_color([1, 0, 0])    # 纯红色
    end_sphere.translate(end_pose[:3, 3])
    
    end_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3.0, origin=[0, 0, 0])
    end_frame.transform(end_pose)
    
    geometries.extend([end_sphere, end_frame])

    # 5. 启动可视化
    print("Green sphere: START | Red sphere: END")
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Trajectory Viewer (Start=Green, End=Red)", width=1280, height=720)
    
    for g in geometries:
        vis.add_geometry(g)
    
    # 获取渲染控制并设置点的大小（如果需要的话）
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0.1, 0.1, 0.1]) # 设置深色背景更易观察坐标轴
    opt.line_width = 2.0
    
    vis.run()
    vis.destroy_window()

visualize_trajectory_open3d(poses, visual_interval=10)

Green sphere: START | Red sphere: END


## 第四步：点云建图（Point Cloud Mapping）

In [ ]:
import os
import numpy as np
import open3d as o3d

# ================= 1. 配置参数 =================
LIDAR_FOLDER = 'os1_cloud_node_kitti_bin'

# [参数设置] 帧间隔 (Step Size)
# 调大这个数字可以减少重叠，方便观察每一帧的结构；调小则地图更密
STEP = 50  

# [参数设置] 处理的总帧数 (设为 None 则处理所有帧)
MAX_FRAMES = 200 

# ================= 2. 辅助函数 =================

def load_lidar_col(bin_path):
    """ 读取点云并转换为齐次坐标列向量 (4, N) """
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    xyz = points[:, :3]
    num_points = xyz.shape[0]
    ones = np.ones((num_points, 1))
    points_h = np.hstack([xyz, ones])
    return points_h.T

def parse_poses(filename):
    """ 读取 poses.txt """
    poses = []
    if not os.path.exists(filename):
        return []
    with open(filename, 'r') as f:
        for line in f:
            values = [float(v) for v in line.strip().split()]
            pose = np.eye(4)
            pose[:3, :4] = np.array(values).reshape(3, 4)
            poses.append(pose)
    return poses

def get_custom_colors(local_points_col):
    """
    根据局部坐标 (x,y,z) 的象限/范围设定颜色
    输入: local_points_col (4, N)
    输出: colors (3, N) - RGB values in [0, 1]
    """
    # 提取局部坐标 x, y, z
    x = local_points_col[0, :]
    y = local_points_col[1, :]
    z = local_points_col[2, :]
    
    N = local_points_col.shape[1]
    
    # 默认初始化为黑色 [0, 0, 0]
    colors = np.zeros((3, N), dtype=np.float64)
    
    # 定义颜色常量
    color_x = np.array([[1.0], [0.0], [0.0]]) # 红色 (对应 X轴)
    color_y = np.array([[0.0], [1.0], [0.0]]) # 绿色 (对应 Y轴)
    color_z = np.array([[0.0], [0.0], [1.0]]) # 蓝色 (对应 Z轴)
    
    # --- 你的自定义逻辑 ---
    
    # 1. x>0 y<0 z<0 -> X轴颜色 (红)
    mask_1 = (x > 0) & (y < 0) & (z < 0)
    colors[:, mask_1] = color_x

    # 2. x<0 y>0 z<0 -> Y轴颜色 (绿)
    mask_2 = (x < 0) & (y > 0) & (z < 0)
    colors[:, mask_2] = color_y
    
    # 3. x<0 y<0 z>0 -> Z轴颜色 (蓝)
    mask_3 = (x < 0) & (y < 0) & (z > 0)
    colors[:, mask_3] = color_z
    
    # 其他情况保持黑色
    
    return colors

# ================= 3. 主逻辑：建图 =================

pose_file = os.path.join(DATA_ROOT, SEQ, 'poses.txt')
lidar_dir = os.path.join(DATA_ROOT, SEQ, LIDAR_FOLDER)
poses = parse_poses(pose_file)

global_points_list = []  # 存位置
global_colors_list = []  # 存颜色
visual_geometries = []   # 存几何体 (Open3D对象)

# 确定循环范围
num_poses = len(poses)
limit = num_poses if MAX_FRAMES is None else min(num_poses, MAX_FRAMES)
frame_indices = range(0, limit, STEP)

print(f"Starting mapping with STEP={STEP}...")

for i in frame_indices:
    # 1. 获取位姿
    T = poses[i]
    
    # 2. 读取点云 (局部系)
    bin_path = os.path.join(lidar_dir, f"{i:06d}.bin")
    if not os.path.exists(bin_path): continue
    
    P_local = load_lidar_col(bin_path) # (4, N)
    
    # 3. [关键步骤] 在变换前计算颜色
    # 原本题目要求是基于“原始的点云坐标”。发现显示的图像不符合直觉。
    # 现在给外变换到雷达系后的点云坐标。
    
    
    # 4. 变换坐标到世界系
    P_world = T @ P_local # (4, N)
    C_local = get_custom_colors(P_world) # (3, N)
    
    # 5. 存储
    global_points_list.append(P_world[:3, :]) # 只存 XYZ
    global_colors_list.append(C_local)
    
    # 6. 添加轨迹上的小坐标轴 (可选，方便看中间过程)
    axis_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0)
    axis_mesh.transform(T)
    visual_geometries.append(axis_mesh)

# ================= 4. 处理起点和终点 (特殊标注) =================

if len(poses) > 0:
    # --- 起点 (Start Node) ---
    start_pose = poses[0]
    
    # 绿色球体
    start_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.8)
    start_sphere.paint_uniform_color([0, 1, 0])  # 纯绿色
    start_sphere.translate(start_pose[:3, 3])
    start_sphere.compute_vertex_normals() # 渲染光照需要
    
    # 大坐标轴
    start_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3.0, origin=[0, 0, 0])
    start_frame.transform(start_pose)
    
    visual_geometries.extend([start_sphere, start_frame])

    # --- 终点 (End Node) ---
    # 如果我们只跑了部分帧 (MAX_FRAMES)，终点应该是最后一帧的 pose
    last_idx = frame_indices[-1]
    end_pose = poses[last_idx]
    
    # 红色球体
    end_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.8)
    end_sphere.paint_uniform_color([1, 0, 0])    # 纯红色
    end_sphere.translate(end_pose[:3, 3])
    end_sphere.compute_vertex_normals()
    
    # 大坐标轴
    end_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3.0, origin=[0, 0, 0])
    end_frame.transform(end_pose)
    
    visual_geometries.extend([end_sphere, end_frame])

# ================= 5. 合并并显示 =================

if global_points_list:
    # 合并数组
    all_points = np.hstack(global_points_list).T # (Total_N, 3)
    all_colors = np.hstack(global_colors_list).T # (Total_N, 3)
    
    print(f"Building PointCloud with {all_points.shape[0]} points...")
    
    # 创建点云对象
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(all_points)
    pcd.colors = o3d.utility.Vector3dVector(all_colors)
    
    # 降采样 (可选，为了流畅度)
    # pcd = pcd.voxel_down_sample(voxel_size=0.1)
    
    visual_geometries.append(pcd)
    
    print("Visualizing... (Red Sphere=End, Green Sphere=Start)")
    o3d.visualization.draw_geometries(visual_geometries,
                                      window_name="Custom Colored Map",
                                      width=1024, height=768)
else:
    print("No data found.")

Starting mapping with STEP=50...
Building PointCloud with 524288 points...
Visualizing... (Red Sphere=End, Green Sphere=Start)
